In [2]:
import sys
sys.path.append('..')

import os
from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
from geopandas.tools import sjoin

from utilities_amigocloud import AmigocloudFunctions

In [3]:
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\SHP\CATASTRO_S13.shp'
PATH_XLSX_GRUPOS = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\DATA\GRUPO_COSECHA.xlsx'

In [4]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_pulverizacion.planificacion_pulv
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.planificacion_pulv
                SET procesado = true
                WHERE id = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def obtener_parte_diario_por_os(ors):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_pulverizacion.parte_diario_pulv where os = {ors}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 35248
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_360912 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['os']}', ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
        """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f'Lote registrado en AmigoCloud')
    return None

In [5]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [6]:
gdf_cat = gpd.read_file(PATH_CAT)
gdf_plan = obtener_planificacion()
gdf_plan

,id,geom,codigo_canero,nombre_canero,obs,procesado
0,51,"MULTIPOINT (465659.079 8077069.785, 465843.719...",30035,GUTIERREZ AGUILERA WIDEN GUILLER,None,False


# CARGAR PARTE DIARIO

In [7]:
gdf_intersect_result = gpd.sjoin(gdf_cat, gdf_plan, how='inner')
os_para_cargar = list(set(gdf_intersect_result['id_right']))
os_para_cargar

[51]

In [8]:
gdf_result = gdf_intersect_result[['geometry', 'unidad_01', 'unidad_02', 'codigo_canero', 'nombre_canero', 'unidad_05', 'area', 'id_right', 'soca', 'zona']].copy()
gdf_result['area'] = gdf_result.geometry.area / 10000
gdf_result.rename(columns={
    'geometry': 'geom',
    'codigo_canero': 'unidad_03',
    'nombre_canero': 'unidad_04',
    'id_right': 'os',
    'zona': 'inst'
}, inplace=True)

gdf_result = gdf_result.set_geometry("geom")
gdf_result['unidad_01'] = gdf_result['unidad_01'].astype(int)
gdf_result['unidad_03'] = gdf_result['unidad_03'].astype(int)
gdf_result['os'] = gdf_result['os'].astype(int)
gdf_result['soca'] = gdf_result['soca'].astype(int)
gdf_result['inst'] = gdf_result['inst'].astype(int)

In [9]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst
3361,"POLYGON ((465509.579 8077157.826, 465508.141 8...",306,ENTRE RIOS--GUTIERREZ WIDEN,30035,GUTIERREZ AGUILERA WIDEN GUILLER,L12.2,5.527511,51,0,50
13335,"POLYGON ((465921.147 8076761.713, 466061.602 8...",306,ENTRE RIOS--GUTIERREZ WIDEN,30035,GUTIERREZ AGUILERA WIDEN GUILLER,L9,10.002896,51,0,50
13419,"POLYGON ((465393.208 8076101.597, 465371.380 8...",306,ENTRE RIOS--GUTIERREZ WIDEN,30035,GUTIERREZ AGUILERA WIDEN GUILLER,L6,14.954778,51,0,50


In [10]:
# GARANTIZAR LA INST DEL CAÑERO
# cargar grupos de cosecha
df_grupos = pd.read_excel(PATH_XLSX_GRUPOS, sheet_name='CODIGOS')
df_grupos = df_grupos[df_grupos['INSTITUCION'].notna()]
df_grupos['CODIGO CAÑERO'] = df_grupos['CODIGO CAÑERO'].astype(int)
df_grupos['INSTITUCION'] = df_grupos['INSTITUCION'].astype(int)

# Crear un diccionario de mapeo: {codigo_cañero: institucion}
mapa_institucion = df_grupos.set_index('CODIGO CAÑERO')['INSTITUCION'].to_dict()
# Reemplazar los valores de 'inst' en gdf_result usando el diccionario
# a partir del diccionario, busca el codigo cañero (unidad_03), y reempalza la institucion
gdf_result['inst'] = gdf_result['unidad_03'].map(mapa_institucion)

In [11]:
gdf_result

,geom,unidad_01,unidad_02,unidad_03,unidad_04,unidad_05,area,os,soca,inst
3361,"POLYGON ((465509.579 8077157.826, 465508.141 8...",306,ENTRE RIOS--GUTIERREZ WIDEN,30035,GUTIERREZ AGUILERA WIDEN GUILLER,L12.2,5.527511,51,0,50
13335,"POLYGON ((465921.147 8076761.713, 466061.602 8...",306,ENTRE RIOS--GUTIERREZ WIDEN,30035,GUTIERREZ AGUILERA WIDEN GUILLER,L9,10.002896,51,0,50
13419,"POLYGON ((465393.208 8076101.597, 465371.380 8...",306,ENTRE RIOS--GUTIERREZ WIDEN,30035,GUTIERREZ AGUILERA WIDEN GUILLER,L6,14.954778,51,0,50


In [12]:
# agregar nuevos registros a la base de datos
gdf_result.to_postgis("parte_diario_pulv", obtener_engine(), schema="drones_pulverizacion", if_exists="append")

In [13]:
os_para_cargar

[51]

In [14]:
# CARGAR A AMIGOCLOD
for i in os_para_cargar:
    plan_os = obtener_parte_diario_por_os(i)
    cargar_a_amigocloud(plan_os)
    marcar_como_procesado(i)

Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
Lote registrado en AmigoCloud
✔️ id 51 marcado como procesado.
